# Модель (*)ARCH (пакет [`sktime`](https://www.sktime.net/en/stable/index.html))  

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arch import ARCH
from sktime.utils.plotting import plot_series
# временной горизонт для прогнозирования
from sktime.forecasting.base import ForecastingHorizon

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
# from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
# warnings.simplefilter('ignore', category=ValueWarning)
# warnings.simplefilter('ignore', category=ConvergenceWarning)

## Подгонка (*)ARCH заданного порядка

Используем класс [`ARCH`](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arch.ARCH.html)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
rate = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')
y = rate.diff().dropna()

y.plot()
plt.show()

Подгоним модель AR(1)-GARCH(1,0,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi_1y_{t-1}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha|\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

где $\lambda=2$

In [ ]:
forecaster = ARCH(mean='AR', lags=1, vol='GARCH', p=1, q=1, power=2)  
forecaster.fit(y)

forecaster.summary()

In [ ]:
forecaster.get_fitted_params()

## Прогнозирование

построим прогноз на 5 периодов по модели AR(2)-GARCH(1,0,1)

Спецификация

\begin{align*}
	y_t&=\alpha_0+\phi_1 y_{t-1}+\phi_2 y_{t-2}+u_t \\ u_t&=\sigma_t \epsilon_t \\
	\sigma^\lambda_t &=\omega+\alpha |\epsilon_{t-1}|^\lambda+\beta\sigma^\lambda_{t-1}
\end{align*}

где $\lambda=2$

In [ ]:
forecaster = ARCH(mean='AR', lags=2, vol='GARCH', p=1, q=1, power=2)

# зададим горизонт прогнозирования и частотность
fh = ForecastingHorizon(np.arange(1,6), freq ='W-Fri')

y_pred = forecaster.fit_predict(y=y, fh=fh)

y_pred

In [ ]:
# прогноз волатильности
forecaster.predict_var(fh=fh)